# Predicción de lluvia en Madrid

## 1. Pre-proceso el dataset

In [43]:
library(keras)

In [44]:
df <- load('Madrid_Alumno.rda') 

In [45]:
df

[1] "xTrain" "xTest"  "yTrain"

In [46]:
n = length(yTrain)
print(n)

[1] 8766


In [47]:
# Preparo la variable de ocurrencia de lluvia
rain = ifelse(yTrain < 1, 0, 1)

# Miro el máximo valor de lluvia, para normalizar la variable 'rain'
# Al final, tengo que acordarme que voy a tener que multiplicar el valor de la predicción en el test para este valor

max.rain <- max(yTrain)
max.rain

# Guardo la predicción original y normalizo
yTrain.original <- yTrain
yTrain <- yTrain / max.rain

[1] 40.6

In [48]:
# Normalizo los datasets

xTrain <- as.data.frame(xTrain)
xTrain.norm <- as.data.frame(scale(xTrain))

head(xTrain.norm)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V441,V442,V443,V444,V445,V446,V447,V448,V449,V450
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.52142098,0.55981865,-0.9171793,0.31395345,0.458461933,0.16233670,1.7499998,1.9117031,-0.4168078,-0.66849173,⋯,-2.0399204,-2.2206501,-1.2493483,-0.8129920,-1.6175884,-1.406849,2.4038027,3.4903915,-1.0879199,-1.7797370
2,0.44727690,0.52952553,-1.0675970,-0.34164908,0.341357533,0.02978809,1.9779085,1.2983166,-0.8249920,-0.76691017,⋯,-2.4796005,-1.9629982,-1.6282727,-1.7329882,-2.7077153,-2.993713,1.5135270,0.5190716,-2.2598730,-2.9433857
3,0.03772347,0.02303596,-0.3881721,-0.54324104,0.009767975,0.02518762,2.1067254,1.8921719,0.2249257,0.19689288,⋯,-1.7036564,-1.1867436,-1.2879048,-1.7353624,-1.9364905,-2.351410,0.9555151,0.2132743,-1.8999771,-1.6280249
4,-1.72335811,-1.89546801,1.3470362,0.43817429,-0.893491576,-1.31972069,2.9865272,2.1883088,1.3729277,0.60364496,⋯,-1.9344421,-2.2020546,-0.1441246,-0.1654673,-1.1110999,-1.441957,0.9892718,0.4346358,0.5381754,1.3849966
5,-2.69341099,-2.70931847,-0.3837467,-0.61381639,-2.148102210,-2.05527774,0.5231868,-0.1918269,1.2800548,0.09570750,⋯,-1.4196737,-1.3610246,-0.6299080,-0.5990789,-1.3390705,-1.250222,0.4977676,-0.4896605,0.6406874,0.2935718
6,-2.10085749,-2.09250052,-0.2745163,0.02094807,-1.851887677,-1.54939245,-0.9790591,-1.6882284,0.7036610,0.07616532,⋯,-0.6490318,-0.3838484,-0.7576004,-0.8280296,-0.9271776,-1.068486,0.4187257,-0.2409056,0.2736218,-0.1886621


In [49]:
# Aplico al dataset the test la misma normalización que al dataset de entrenamiento
# https://datascience.stackexchange.com/questions/13971/standardization-normalization-test-data-in-r

trainMean <- apply(xTrain, 2, mean)
trainSd   <- apply(xTrain, 2, sd)

xTest <- as.data.frame(xTest)
xTest.norm <- sweep(sweep(xTest, 2L, trainMean), 2, trainSd, "/")

head(xTest.norm)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V441,V442,V443,V444,V445,V446,V447,V448,V449,V450
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.6379673,0.75290759,0.5917206,-0.3041471,0.3274740,0.3539039,1.776219,1.141768,-0.05714004,0.6405975,⋯,-0.6199747,-0.5987160,-0.8408304,-0.3246940,-0.61372068,-0.49936470,0.4339928,0.5607600,-1.325054840,-1.0292307
2,0.4183818,0.35939736,-0.3388903,-0.8832639,0.3779006,0.4678298,2.717766,2.629239,1.54727554,1.5466540,⋯,-0.3904226,-0.5705003,0.4386167,0.2276853,-0.02763776,-0.02703828,2.2295538,2.4968296,-0.261390879,-0.3600638
3,-0.2100928,-0.05317012,-0.4410847,0.1411624,-0.3854247,-0.5702071,2.501309,2.047083,1.90339017,1.2995759,⋯,-0.9842224,-1.0610985,-0.1669691,-0.3682557,-0.66397103,-0.64334852,3.0078470,2.8665486,0.001767713,-0.3305455
4,-0.4851375,-0.48701398,-0.5224260,0.2096232,-0.2186639,-0.8908143,1.758244,1.437589,0.20093977,0.4174770,⋯,-1.5156033,-1.4554449,-1.2647502,-0.9530235,-1.50813359,-1.18717745,1.6825174,2.2316662,0.177076886,-0.4431701
5,-0.7594815,-0.98485288,0.2194023,-0.5108478,-0.2495626,-0.4749695,1.506337,1.672491,-0.65396791,-0.3565761,⋯,-2.3114263,-2.3661289,-1.6627814,-1.5211448,-1.64008857,-1.86503976,1.3696095,0.5465005,-0.955277482,-1.2527752
6,-0.8949669,-1.14205794,1.0160999,-0.2163544,-0.1931065,-0.8038468,1.491730,1.767074,-1.28217139,-1.3758111,⋯,-1.9686848,-2.3484660,-0.1829465,-0.7520126,-0.99202781,-1.32116757,-0.1492214,-1.0151496,0.746333377,0.7311858


In [50]:
# Dataset de entrenamiento
indtrain   <- sample(1:n, round(0.8*n)) 
x_train    <- xTrain.norm[indtrain,]
y_train    <- yTrain[indtrain]
rain_train <- rain[indtrain]
train      <- x_train

# Añado las variables objetivo
train['y'] <- y_train
train['rain'] <- as.factor(rain_train)

# Miro el dataset
head(train)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V443,V444,V445,V446,V447,V448,V449,V450,y,rain
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
4837,-1.5280019,-1.1448833,-0.8350705,-0.84536043,-1.69407959,-1.593326695,0.28816630,0.2243921,-0.739578173,-0.82419002,⋯,-0.7069790,-0.91360995,-0.9097027,-0.7706383,0.006267466,-0.04508313,-0.1721817,-0.2811957,0.004926108,0
7092,-0.1836921,-0.4634355,0.7874556,0.53970766,0.32520107,0.002304575,1.17043057,1.6275111,1.682248996,1.49515114,⋯,1.5275861,0.24647985,0.7811043,0.4081309,1.300304804,1.11843390,0.6099672,0.7233030,0.009852217,0
4294,0.3212973,0.1306190,0.0181804,0.03732943,0.58201882,0.373886997,0.85527145,0.9874634,0.073580075,0.07875236,⋯,0.7774192,1.22559817,0.8501534,0.5377569,-0.588316972,-0.22521982,0.7238174,1.1277845,0.000000000,0
1749,0.6465509,0.9305683,-1.1908413,-0.73419058,0.08662958,0.259251010,0.02191298,0.9750341,0.014646038,-0.01116063,⋯,-0.2695006,0.15357263,0.3335341,0.1987716,1.048698070,1.51782389,0.7619404,0.8443639,0.000000000,0
932,1.4318173,1.0735891,-0.8817578,-1.30744427,1.31048042,1.953216710,-0.90082800,-0.5973410,0.336151723,0.41226724,⋯,1.1857704,0.07071533,1.1177060,0.7030746,-0.071744710,-0.36992740,-1.8623132,-1.4909533,0.000000000,0
3336,-1.1384550,-0.8991182,-0.8653689,-0.45297409,-1.48497992,-1.213885873,0.33837001,-0.3855049,-0.006856281,0.08299876,⋯,-0.9362058,-0.84671945,-0.3048959,0.1956297,-1.002058943,-0.99630975,-0.8539764,-1.0179929,0.000000000,0


In [51]:
# Dataset de validación
indvalid   <- setdiff(1:n, indtrain)      
x_valid    <- as.data.frame(xTrain[indvalid,])
y_valid    <- yTrain[indvalid]
rain_valid <- rain[indvalid]
valid      <- x_valid

# Añado las variables objetivo
valid['y']    <- y_valid
valid['rain'] <- as.factor(rain_valid)

# Miro el dataset
head(train)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V443,V444,V445,V446,V447,V448,V449,V450,y,rain
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
4837,-1.5280019,-1.1448833,-0.8350705,-0.84536043,-1.69407959,-1.593326695,0.28816630,0.2243921,-0.739578173,-0.82419002,⋯,-0.7069790,-0.91360995,-0.9097027,-0.7706383,0.006267466,-0.04508313,-0.1721817,-0.2811957,0.004926108,0
7092,-0.1836921,-0.4634355,0.7874556,0.53970766,0.32520107,0.002304575,1.17043057,1.6275111,1.682248996,1.49515114,⋯,1.5275861,0.24647985,0.7811043,0.4081309,1.300304804,1.11843390,0.6099672,0.7233030,0.009852217,0
4294,0.3212973,0.1306190,0.0181804,0.03732943,0.58201882,0.373886997,0.85527145,0.9874634,0.073580075,0.07875236,⋯,0.7774192,1.22559817,0.8501534,0.5377569,-0.588316972,-0.22521982,0.7238174,1.1277845,0.000000000,0
1749,0.6465509,0.9305683,-1.1908413,-0.73419058,0.08662958,0.259251010,0.02191298,0.9750341,0.014646038,-0.01116063,⋯,-0.2695006,0.15357263,0.3335341,0.1987716,1.048698070,1.51782389,0.7619404,0.8443639,0.000000000,0
932,1.4318173,1.0735891,-0.8817578,-1.30744427,1.31048042,1.953216710,-0.90082800,-0.5973410,0.336151723,0.41226724,⋯,1.1857704,0.07071533,1.1177060,0.7030746,-0.071744710,-0.36992740,-1.8623132,-1.4909533,0.000000000,0
3336,-1.1384550,-0.8991182,-0.8653689,-0.45297409,-1.48497992,-1.213885873,0.33837001,-0.3855049,-0.006856281,0.08299876,⋯,-0.9362058,-0.84671945,-0.3048959,0.1956297,-1.002058943,-0.99630975,-0.8539764,-1.0179929,0.000000000,0


In [52]:
# Dataset de test (solo tengo la x, voy a producir la y en el ejercicio)
test <- xTest.norm
head(test)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V441,V442,V443,V444,V445,V446,V447,V448,V449,V450
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.6379673,0.75290759,0.5917206,-0.3041471,0.3274740,0.3539039,1.776219,1.141768,-0.05714004,0.6405975,⋯,-0.6199747,-0.5987160,-0.8408304,-0.3246940,-0.61372068,-0.49936470,0.4339928,0.5607600,-1.325054840,-1.0292307
2,0.4183818,0.35939736,-0.3388903,-0.8832639,0.3779006,0.4678298,2.717766,2.629239,1.54727554,1.5466540,⋯,-0.3904226,-0.5705003,0.4386167,0.2276853,-0.02763776,-0.02703828,2.2295538,2.4968296,-0.261390879,-0.3600638
3,-0.2100928,-0.05317012,-0.4410847,0.1411624,-0.3854247,-0.5702071,2.501309,2.047083,1.90339017,1.2995759,⋯,-0.9842224,-1.0610985,-0.1669691,-0.3682557,-0.66397103,-0.64334852,3.0078470,2.8665486,0.001767713,-0.3305455
4,-0.4851375,-0.48701398,-0.5224260,0.2096232,-0.2186639,-0.8908143,1.758244,1.437589,0.20093977,0.4174770,⋯,-1.5156033,-1.4554449,-1.2647502,-0.9530235,-1.50813359,-1.18717745,1.6825174,2.2316662,0.177076886,-0.4431701
5,-0.7594815,-0.98485288,0.2194023,-0.5108478,-0.2495626,-0.4749695,1.506337,1.672491,-0.65396791,-0.3565761,⋯,-2.3114263,-2.3661289,-1.6627814,-1.5211448,-1.64008857,-1.86503976,1.3696095,0.5465005,-0.955277482,-1.2527752
6,-0.8949669,-1.14205794,1.0160999,-0.2163544,-0.1931065,-0.8038468,1.491730,1.767074,-1.28217139,-1.3758111,⋯,-1.9686848,-2.3484660,-0.1829465,-0.7520126,-0.99202781,-1.32116757,-0.1492214,-1.0151496,0.746333377,0.7311858
